# Preprocess data for DL models

Simple preprocessing pipeline for the DL models data.

- Load raw EEG data from files;
- Load metadata from files (ages of subjects);
- Apply filters (bandpass);
- Detect potential bad channels and replace them by interpolation;
- Detect potential bad epochs and remove them;
- Save processed data and metadata.

## Import packages & links

In [1]:
# Import packages
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import h5py
import pandas as pd
import mne
import zarr

C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:321: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(scipy.__version__) >= '1.1':
C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):
C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):


In [2]:
from config import PATH_RAW_DATA, PATH_METADATA, PATH_DATA_PROCESSED_DL

## Search all *.cnt files and get paths, code, and age group

Plug in new data as data files to run on

In [3]:

print(PATH_RAW_DATA)

C:/Projects\EEG_explorer\Data_Old


In [4]:
# sorted_df = pd.DataFrame(file_name_list, columns=["file_name"])
# sorted_df['has_month'] = sorted_df["file_name"].str.contains('17m') | sorted_df["file_name"].str.contains('29m')
# sorted_df

In [5]:
# # show me classes on df?
# 
# for p in dir(sorted_df["file_name"].str):
#     print(p)

In [6]:
# OK, let's shortcut to just ones where I am sure of the age:

# aged_df = sorted_df[sorted_df.has_month]
# #aged_df 

In [7]:
## make me some local dirs
# os.mkdir('17mnd_mmn')
# os.mkdir('29mnd_mmn')

In [8]:
PATH_RAW_DATA

'C:/Projects\\EEG_explorer\\Data_Old'

In [9]:
# for file in aged_df.file_name:
#     copy file into directory according to which label it has
import glob
import shutil
from pathlib import Path


old_eegs_file_root = "C:/Projects/EEG_explorer/Data_Old/"

pattern = os.path.join(PATH_RAW_DATA, '*.cnt')

originals = glob.glob(pattern, recursive=True)
src = old_eegs_file_root
trg  = '17mnd_mmn'
trg2 = '29mnd_mmn'

files1= []

for file in originals:
    if '17m' in file:
        shutil.copy2(os.path.join(src,file), trg)
    if '29m' in file:
        shutil.copy2(os.path.join(src,file), trg2)


In [10]:
import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions



dir_names = {#11: '11mnd mmn',
             17: '17mnd_mmn',
             #23: '23mnd mmn',
             29: '29mnd_mmn',
             #35: '35mnd mmn',
             #41: '41mnd mmn',
             # 47: '47mnd mmn'
}
    
df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = directory
    file_names = os.listdir(dir_path)
    cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
    cnt_files = [x.split('/')[-1][:-4] for x in cnt_paths]
    #codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    df = pd.DataFrame(list(zip( cnt_paths, cnt_files)), columns =[ 'cnt_path','cnt_file']) 
    
    df['age_group'] = age_group
    df_list.append(df)

cnt_files = pd.concat(df_list)

In [11]:
dir_names.items()

dict_items([(17, '17mnd_mmn'), (29, '29mnd_mmn')])

# now we can figure out the code by matching them to files on Bjorn's dataset
we will make two tables and compare hashes

In [12]:
import new_core_helper_functions

good_files = new_core_helper_functions.hash_it_up_right_all('17mnd_mmn','.cnt')
good_files

,file_name,hash
0,17mnd_mmn\602-115-17m-mc-mmn.cnt,6285bba3aadba0a2efa5d2c941eb193a3a66223c5caba3...
1,17mnd_mmn\604-133-17m-jc-mmn36.cnt,4277f04e7f14a65c68bb768cb056e26c8e24b32c166d56...
2,17mnd_mmn\605-131-17m-jc-mmn.cnt,10679da12409265f60f538959ae84751c5fcb9b77ee513...
3,17mnd_mmn\607-000-17m-jc-mmn1_36.cnt,2c7923bf82ad1a4ba577225a1e60d78d4f99e96a3c609e...
4,17mnd_mmn\608-170-17m-mc-mmn36_2.cnt,7c574edb7deb4c687ad79afabaf0cc67215f7d41217909...
...,...,...
59,17mnd_mmn\753-470-17m-mr-mmn36.cnt,ccfd67842b21e618ab05ac2842f847a8f3c2b799eb01ad...
60,17mnd_mmn\754-472-17m-jr-mmn36.cnt,70c42b135d47b99d064fb2cdc3ca99ca24b12053e06007...
61,17mnd_mmn\755-471-17m-mr-mmn36.cnt,341fa6f10a938dedd23ae599de36f6bb0fa6b2a90d211f...
62,17mnd_mmn\757-487-17m-jr-mmn36.cnt,dd72207c7fbda99fd3fa6969aace4410ced7342cb51202...


In [13]:
# make old Bjorn dataset from files on drive
# bjorn_directory=  'C:\Projects\EEG_explorer\Data\17mnd mmn'

old_files = new_core_helper_functions.hash_it_up_right_all('C:/Projects/EEG_explorer/Data/17mnd mmn','.cnt')
old_files

,file_name,hash
0,C:/Projects/EEG_explorer/Data/17mnd mmn\001_17...,b2e20730220ee9f2a1b2c1a1a373c22555e616ba66a56d...
1,C:/Projects/EEG_explorer/Data/17mnd mmn\005_17...,d37678cbfd0d1337a2423deb3d513001e8b4a3c234bf03...
2,C:/Projects/EEG_explorer/Data/17mnd mmn\006_17...,c09f38853700fba99e1fd2b204d67b9775453ba5befbf0...
3,C:/Projects/EEG_explorer/Data/17mnd mmn\007_17...,b86e2a72ade7393ad90f4fee9b9fed2d127c50388eb0ed...
4,C:/Projects/EEG_explorer/Data/17mnd mmn\007_17...,87f473e8d7a40bc65f93036da23372799b17d6db59bb72...
...,...,...
235,C:/Projects/EEG_explorer/Data/17mnd mmn\345_17...,25b4159ea368540c44b6d480f20f887725cbf4966ef933...
236,C:/Projects/EEG_explorer/Data/17mnd mmn\345_17...,70d4b9d4dc036cf5e9b6bf11b8857918e0d05593e87464...
237,C:/Projects/EEG_explorer/Data/17mnd mmn\346_17...,57fd6574fb54e9b4533efefef703afbfc0d56fe3c1a591...
238,C:/Projects/EEG_explorer/Data/17mnd mmn\348_17...,1e807d72ea70963e5b067fa2d935dcea4cfe5120c1bb5c...


In [14]:
# compute and add codes on old files

import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions

#bjorn_directory=  'C:/Projects/EEG_explorer/Data/17mnd mmn'


dir_names = {#11: '11mnd mmn',
             17: '17mnd_mmn',
             #23: '23mnd mmn',
             #29: '29mnd_mmn',
             #35: '35mnd mmn',
             #41: '41mnd mmn',
             # 47: '47mnd mmn'
}
    

df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = directory
    file_names = os.listdir(dir_path)
    cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
    #cnt_files1 = [x.split('/')[-1][:-4] for x in cnt_paths]
    cnt_files = [x.split('/')[-1][10:-4] for x in cnt_paths]
    codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    df = pd.DataFrame(list(zip( codes, cnt_paths, cnt_files)), columns =['code', 'cnt_path','cnt_file']) 
    
    df['age_group'] = age_group
    df_list.append(df)

code_files = pd.concat(df_list)

In [15]:
code_files

,code,cnt_path,cnt_file,age_group
0,602,17mnd_mmn\602-115-17m-mc-mmn.cnt,602-115-17m-mc-mmn,17
1,604,17mnd_mmn\604-133-17m-jc-mmn36.cnt,604-133-17m-jc-mmn36,17
2,605,17mnd_mmn\605-131-17m-jc-mmn.cnt,605-131-17m-jc-mmn,17
3,607,17mnd_mmn\607-000-17m-jc-mmn1_36.cnt,607-000-17m-jc-mmn1_36,17
4,608,17mnd_mmn\608-170-17m-mc-mmn36_2.cnt,608-170-17m-mc-mmn36_2,17
...,...,...,...,...
59,753,17mnd_mmn\753-470-17m-mr-mmn36.cnt,753-470-17m-mr-mmn36,17
60,754,17mnd_mmn\754-472-17m-jr-mmn36.cnt,754-472-17m-jr-mmn36,17
61,755,17mnd_mmn\755-471-17m-mr-mmn36.cnt,755-471-17m-mr-mmn36,17
62,757,17mnd_mmn\757-487-17m-jr-mmn36.cnt,757-487-17m-jr-mmn36,17


In [16]:
# code_files['file_name'] = code_files.cnt_path
# full_code_files =code_files.merge(old_files, on = 'file_name', how= 'right')
# full_code_files.tail()

In [17]:
goodies = good_files.merge(old_files, on = 'hash')
goodies

,file_name_x,hash,file_name_y


## Search for all age files and create DataFrame containing all ages per subject code

In [18]:
# age_files = {5: 'ages_5mnths.txt',
#              11: 'ages_11mnths.txt',
#              17: 'ages_17mnths.txt',
#              23: 'ages_23mnths.txt',
#              29: 'ages_29mnths.txt',
#              35: 'ages_35mnths.txt',
#              41: 'ages_41mnths.txt',
#              47: 'ages_47mnths.txt'}

age_files = {#11: 'ages_11mnths.txt',
             17: 'ages_17mnths.txt',
             #23: 'ages_23mnths.txt',
             29: 'ages_29mnths.txt',
             #35: 'ages_35mnths.txt',
             #41: 'ages_41mnths.txt',
             #47: 'ages_47mnths.txt'
}

df_list = []

for age_group, age_file in age_files.items():
    df = pd.read_csv(os.path.join(PATH_METADATA, 'ages', age_file), sep="\t")
    df['age_group'] = age_group
    df_list.append(df)

age_df = pd.concat(df_list)

In [19]:

PATH_METADATA

'C:/Projects\\EEG_explorer\\ePODIUM_Metadata'

In [20]:
age_df

,code,age_days,age_months,age_years,age_group
0,3,524,17.466667,1.455556,17
1,5,507,16.900000,1.408333,17
2,7,528,17.600000,1.466667,17
3,8,512,17.066667,1.422222,17
4,9,514,17.133333,1.427778,17
...,...,...,...,...,...
269,754,874,29.133333,2.427778,29
270,755,862,28.733333,2.394444,29
271,756,872,29.066667,2.422222,29
272,757,875,29.166667,2.430556,29


### Merge the .cnt files with the age information we have on the subjects

In [21]:
merged_df = pd.merge(code_files, age_df, how='left', on=['age_group', 'code'])

In [22]:
merged_df.tail()

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
59,753,17mnd_mmn\753-470-17m-mr-mmn36.cnt,753-470-17m-mr-mmn36,17,512,17.066667,1.422222
60,754,17mnd_mmn\754-472-17m-jr-mmn36.cnt,754-472-17m-jr-mmn36,17,510,17.000000,1.416667
61,755,17mnd_mmn\755-471-17m-mr-mmn36.cnt,755-471-17m-mr-mmn36,17,511,17.033333,1.419444
62,757,17mnd_mmn\757-487-17m-jr-mmn36.cnt,757-487-17m-jr-mmn36,17,524,17.466667,1.455556
63,758,17mnd_mmn\758-465-17m-mr-mmn36.cnt,758-465-17m-mr-mmn36,17,510,17.000000,1.416667


In [23]:
merged_df.head(20)

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,602,17mnd_mmn\602-115-17m-mc-mmn.cnt,602-115-17m-mc-mmn,17,512,17.066667,1.422222
1,604,17mnd_mmn\604-133-17m-jc-mmn36.cnt,604-133-17m-jc-mmn36,17,510,17.000000,1.416667
2,605,17mnd_mmn\605-131-17m-jc-mmn.cnt,605-131-17m-jc-mmn,17,510,17.000000,1.416667
3,607,17mnd_mmn\607-000-17m-jc-mmn1_36.cnt,607-000-17m-jc-mmn1_36,17,508,16.933333,1.411111
4,608,17mnd_mmn\608-170-17m-mc-mmn36_2.cnt,608-170-17m-mc-mmn36_2,17,512,17.066667,1.422222
5,609,17mnd_mmn\609-158-17m-jc-mmn36.cnt,609-158-17m-jc-mmn36,17,510,17.000000,1.416667
6,610,17mnd_mmn\610_185_17m_jc_mmn36.cnt,610_185_17m_jc_mmn36,17,511,17.033333,1.419444
7,611,17mnd_mmn\611_157_17m_mc_mmn36.cnt,611_157_17m_mc_mmn36,17,514,17.133333,1.427778
8,613,17mnd_mmn\613-176-17m-mc-mmn36.cnt,613-176-17m-mc-mmn36,17,517,17.233333,1.436111
9,618,17mnd_mmn\618-163-17m-j-c-mmn36.cnt,618-163-17m-j-c-mmn36,17,520,17.333333,1.444444


In [24]:
len(merged_df)

64

## Check files with no label

In [25]:
merged_df.loc[merged_df['age_days'].isnull()]

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years


### Fill in the missing age data based on the age group the subject is in

We know the age group (i.e. 11, 17, 23, .. months etc) of all the subjects, based on the folder the files are in and based on the file name. We have got the exact ages (in days) of most subjects seperately, which we have added to the DataFrame above. For some of the subjects, we don't have the exact age and therefore we set this equal to the age group.

In [26]:
merged_df['age_months'].fillna(merged_df['age_group'], inplace=True)
merged_df['age_days'].fillna(merged_df['age_group']*30, inplace=True)
merged_df['age_years'].fillna(merged_df['age_group']/12, inplace=True)

Below should now return an empty dataframe, because all empty fields have been filled

In [27]:
merged_df.loc[merged_df['age_days'].isnull()]

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years


## Helper functions for processing

### Custom cnt-file import function and data standardize function:

In [28]:
def read_cnt_file(file,
                  label_group,
                  event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                  channel_set = "30",
                  tmin = -0.2,
                  tmax = 0.8,
                  lpass = 0.5, 
                  hpass = 40, 
                  threshold = 5, 
                  max_bad_fraction = 0.2,
                  max_bad_channels = 2):
    """ Function to read cnt file. Run bandpass filter. 
    Then detect and correct/remove bad channels and bad epochs.
    Store resulting epochs as arrays.
    
    Args:
    --------
    file: str
        Name of file to import.
    label_group: int
        Unique ID of specific group (must be >0).
    channel_set: str
        Select among pre-defined channel sets. Here: "30" or "62"
    """
    
    if channel_set == "30":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1']
    elif channel_set == "62":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1', 'AFZ', 'PO3', 
                       'P1', 'POZ', 'P2', 'PO4', 'CP2', 'P6', 'M1', 'CP6', 'C6', 'PO8', 'PO7', 
                       'P5', 'CP5', 'CP1', 'C1', 'C2', 'FC2', 'FC6', 'C5', 'FC1', 'F2', 'F6', 
                       'FC5', 'F1', 'AF4', 'AF8', 'F5', 'AF7', 'AF3', 'FPZ']
    else:
        print("Predefined channel set given by 'channel_set' not known...")
        
    
    # Initialize array
    signal_collection = np.zeros((0,len(channel_set),501))
    label_collection = [] #np.zeros((0))
    channel_names_collection = []
    
    # Import file
    try:
        data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
    except ValueError:
        print("ValueError")
        print("Could not load file:", file)
        return None, None, None
    
    # Band-pass filter (between 0.5 and 40 Hz. was 0.5 to 30Hz in Stober 2016)
    data_raw.filter(0.5, 40, fir_design='firwin')

    # Get events from annotations in the data
    events_from_annot, event_dict = mne.events_from_annotations(data_raw)
    
    # Set baseline:
    baseline = (None, 0)  # means from the first instant to t = 0

    # Select channels to exclude (if any)
    channels_exclude = [x for x in data_raw.ch_names if x not in channel_set]
    channels_exclude = [x for x in channels_exclude if x not in ['HEOG', 'VEOG']]
    
    for event_id in event_idx:
        if str(event_id) in event_dict:
            # Pick EEG channels
            picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                               #exclude=data_exclude)#'bads'])
                                   include=channel_set, exclude=channels_exclude)#'bads'])

            epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                baseline=baseline, preload=True, event_repeated='merge', verbose=False)

            # Detect potential bad channels and epochs
            bad_channels, bad_epochs = helper_functions.select_bad_epochs(epochs,
                                                                          event_id,
                                                                          threshold = threshold,
                                                                          max_bad_fraction = max_bad_fraction)

            # Interpolate bad channels
            # ------------------------------------------------------------------
            if len(bad_channels) > 0:
                if len(bad_channels) > max_bad_channels:
                    print(20*'--')
                    print("Found too many bad channels (" + str(len(bad_channels)) + ")")
                    return None, None, None
                else:
                    montage = mne.channels.make_standard_montage('standard_1020')
                    montage.ch_names = [ch_name.upper() for ch_name in montage.ch_names]
                    data_raw.set_montage(montage)
                    
                    # MARK: Think about using all channels before removing (62 -> 30), to enable for better interpolation
                    
                    # Mark bad channels:
                    data_raw.info['bads'] = bad_channels
                    # Pick EEG channels:
                    picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                                       #exclude=data_exclude)#'bads'])
                                       include=channel_set, exclude=channels_exclude)#'bads'])
                    epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                        tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                        baseline=baseline, preload=True, verbose=False)
                    
                    # Interpolate bad channels using functionality of 'mne'
                    epochs.interpolate_bads()
                    

            # Get signals as array and add to total collection
            channel_names_collection.append(epochs.ch_names)
            signals_cleaned = epochs[str(event_id)].drop(bad_epochs).get_data()
            signal_collection = np.concatenate((signal_collection, signals_cleaned), axis=0)
            label_collection += [event_id + label_group] * signals_cleaned.shape[0]

    return signal_collection, label_collection, channel_names_collection

In [29]:
"""Functions to import and process EEG data from cnt files.
"""
def standardize_EEG(data_array,
                    std_aim = 1,
                    centering = 'per_channel',
                    scaling = 'global'):
    """ Center data around 0 and adjust standard deviation.

    Args:
    --------
    data_array: np.ndarray
        Input data.
    std_aim: float/int
        Target standard deviation for rescaling/normalization.
    centering: str
        Specify if centering should be done "per_channel", or "global".
    scaling: str
        Specify if scaling should be done "per_channel", or "global".
    """
    if centering == 'global':
        data_mean = data_array.mean()

        # Center around 0
        data_array = data_array - data_mean

    elif centering == 'per_channel':
        for i in range(data_array.shape[1]):

            data_mean = data_array[:,i,:].mean()

            # Center around 0
            data_array[:,i,:] = data_array[:,i,:] - data_mean

    else:
        print("Centering method not known.")
        return None

    if scaling == 'global':
        data_std = data_array.std()

        # Adjust std to std_aim
        data_array = data_array * std_aim/data_std

    elif scaling == 'per_channel':
        for i in range(data_array.shape[1]):

            data_std = data_array[:,i,:].std()

            # Adjust std to std_aim
            data_array[:,i,:] = data_array[:,i,:] * std_aim/data_std
    else:
        print("Given method is not known.")
        return None

    return data_array

# Preprocessing raw

In [30]:
PATH_DATA_PROCESSED_DL

'C:/Projects\\EEG_explorer\\Data/data_processed_DL/'

In [31]:
import re
from itertools import islice

for i, file in islice(merged_df.iterrows(), 0, None): # Enables you to set starting row of DataFrame
    print(f"Checking out file: {file['cnt_file']}")
        
    filename_processed_raw = os.path.join(PATH_DATA_PROCESSED_DL, "processed_raw" + file['cnt_file'] + ".npy")
    filename_metadata = os.path.join(PATH_DATA_PROCESSED_DL, "processed_metadata_" + file['cnt_file'] + ".csv")
    
    # Step 1: Check if file don't already exist, else skip
    if os.path.exists(filename_processed_raw) and os.path.exists(filename_metadata):
        print(f"Skipping because {filename_processed_raw} and {filename_metadata} already exist.")
        continue
    
    
    # Step 2: Import data and labels 
    signal_collect, label_collect, ch_names = read_cnt_file(file['cnt_path'],
                                                            file['age_months'],
                                                            event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                                                            channel_set = "30",
                                                            tmin = -0.2,
                                                            tmax = 0.8,
                                                            lpass = 0.5, 
                                                            hpass = 40, 
                                                            threshold = 5, 
                                                            max_bad_fraction = 0.2,
                                                            max_bad_channels = 2)    
    
    if signal_collect is None:
        continue
        
    # Step 3: standardize data
    if signal_collect is not None:
        signal_collect = standardize_EEG(signal_collect,
                                         std_aim = 1,
                                         centering = 'per_channel',
                                         scaling = 'global')

    # Step 4: Save raw data and metadata
    if signal_collect is not None:
        np.save(filename_processed_raw, signal_collect)
        pd.DataFrame(file).transpose().to_csv(filename_metadata, sep=',', index=False, header=True)     
    
    # TODO: 1. Try saving 2. Retry after n seconds if fails 3. If fails m times, continue

Checking out file: 602-115-17m-mc-mmn
Used Annotations descriptions: ['0', '12', '13', '2', '3', '55', '66']
Found 5 bad epochs in a total of 11  channels.
Marked 5 bad epochs in a total of 800  epochs.
Found 22 bad epochs in a total of 12  channels.
Marked 22 bad epochs in a total of 800  epochs.
Found 4 bad epochs in a total of 3  channels.
Marked 4 bad epochs in a total of 100  epochs.
Found 2 bad epochs in a total of 3  channels.
Marked 2 bad epochs in a total of 100  epochs.
Checking out file: 604-133-17m-jc-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 115 bad epochs in a total of 30  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  8
Marked 50 bad epochs in a total of 400  epochs.
Found 6 bad epochs in a total of 30  channels.
Marked 6 bad epochs in a total of 50  epochs.
Checking out file: 605-131-17m-jc-mmn
Used Annotations descriptions: ['0', '13', '3', '66']
Found 9 bad epochs in a total of 16  channels.
Marked 9 bad epochs in a tota

Found 168 bad epochs in a total of 30  channels.
Marked 168 bad epochs in a total of 800  epochs.
Found 26 bad epochs in a total of 27  channels.
Marked 26 bad epochs in a total of 100  epochs.
Checking out file: 640-464-17m-jc-mmn36-2
Used Annotations descriptions: ['13', '3', '66']
Found 7 bad epochs in a total of 9  channels.
Marked 7 bad epochs in a total of 800  epochs.
Found 2 bad epochs in a total of 1  channels.
Marked 2 bad epochs in a total of 100  epochs.
Checking out file: 640-464-17m-jc-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 3 bad epochs in a total of 7  channels.
Marked 3 bad epochs in a total of 164  epochs.
Found 1 bad epochs in a total of 3  channels.
Marked 1 bad epochs in a total of 22  epochs.
Checking out file: 642-485-17m-jc-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 73 bad epochs in a total of 21  channels.
Marked 73 bad epochs in a total of 800  epochs.
Found 7 bad epochs in a total of 23  channels.
Marked 7 bad 

Checking out file: 743-475-17m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 153 bad epochs in a total of 26  channels.
Marked 153 bad epochs in a total of 1200  epochs.
Found 21 bad epochs in a total of 9  channels.
Marked 21 bad epochs in a total of 150  epochs.
Checking out file: 745-466-17m-mr-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 114 bad epochs in a total of 23  channels.
Marked 114 bad epochs in a total of 800  epochs.
Found 19 bad epochs in a total of 10  channels.
Marked 19 bad epochs in a total of 100  epochs.
Checking out file: 746-438-17m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 8 bad epochs in a total of 16  channels.
Marked 8 bad epochs in a total of 637  epochs.
Found 3 bad epochs in a total of 5  channels.
Marked 3 bad epochs in a total of 80  epochs.
Checking out file: 747-410-17m-mr-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 112 bad epochs in a total of 14  channels.
Marked 112 

## Transform .npy to .zarr

In [32]:
%%time

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_DL)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(file_name)[0] for file_name in fnmatch.filter(file_names, "*.npy")]

for file in files:
    # Step 0: Get the NumPy file name
    np_name = file + ".npy"

    # Step 1: Load existing NumPy array
    X = np.load(os.path.join(PATH_DATA_PROCESSED_DL, np_name))

    # Step 2: Open a Zarr file with the same name and assign the number of epochs as chunks to the file
    zarr_name = np_name.replace(".npy", ".zarr")
    z_file =  zarr.open(os.path.join(PATH_DATA_PROCESSED_DL, zarr_name), 
                        mode='w', 
                        shape=X.shape, 
                        chunks=(1, X.shape[1], X.shape[2]))
    z_file[:] = X

CPU times: total: 45.4 s
Wall time: 51.7 s


## TODO: Remove files that have len(npy array) == 0

We now do this afterwards